In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch
import os
import random
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from numpy.random import rand, randn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from AffectivNetDataloader import AffectivNetDataset
from torchvision import transforms, utils
from torch.utils.data import DataLoader

In [ ]:
transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize((227,227)), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
train_set = AffectivNetDataset(root_dir='C:\\Projects\\AffectivNet', csv_file = 'C:\\Projects\\AffectivNet\\clearedTraining.csv', transform=transform, downsample=10000, delete_classes=[4,5,7], check=False)
test_set = AffectivNetDataset(root_dir='C:\\Projects\\AffectivNet', csv_file = 'C:\\Projects\\AffectivNet\\clearedValidation.csv', transform=transform, delete_classes=[4,5,7])
train = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4)
test = DataLoader(test_set, batch_size=64, shuffle=True, num_workers=4)

In [ ]:
len(train_set.target)

In [ ]:
len(train)

In [ ]:
alexnet = models.alexnet(pretrained=True)
alexnet.classifier._modules['6'] = nn.Linear(4096,5)

In [ ]:
one_layer_gnet_acc_train = []
one_layer_gnet_loss_train = []
one_layer_gnet_acc_test = []
one_layer_gnet_loss_test = []

two_layer_gnet_acc_train = []
two_layer_gnet_loss_train = []
two_layer_gnet_acc_test = []
two_layer_gnet_loss_test = []

convnet_acc_train = []
convnet_loss_train = []
convnet_acc_test = []
convnet_loss_test = []

one_layer_cgnet_acc_train = []
one_layer_cgnet_loss_train = []
one_layer_cgnet_acc_test = []
one_layer_cgnet_loss_test = []

two_layer_cgnet_acc_train = []
two_layer_cgnet_loss_train = []
two_layer_cgnet_acc_test = []
two_layer_cgnet_loss_test = []

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net = alexnet.to(device)
print(net)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
net(torch.normal(torch.Tensor(1,3,227 ,227 )).to(device))

In [ ]:
for epoch in range(10):

    running_loss = 0.0
    correct = 0
    net.train()
    for i, data in enumerate(train, 0):
        # get the inputs
        inputs, labels = data['image'], data['target']

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.to(device))
        labels = labels.type(torch.LongTensor)
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        pred = outputs.max(1, keepdim=True)[1].to('cpu')
        correct += pred.eq(labels.view_as(pred)).sum().item()
        if i != 0 and i % 150 == 0:
            print('#', end = '')

        # print statistics
        running_loss += loss.item()
    print('\n[%d] train_acc: %.3f train_loss: %.3f' %(epoch + 1, correct/len(train_set), running_loss/len(train_set))) 
    one_layer_gnet_acc_train.append(correct/len(train_set))
    one_layer_gnet_loss_train.append(running_loss/len(train_set))
    
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        net.eval()
        for i, data in enumerate(test, 0):
            # get the inputs
            inputs, labels = data['image'], data['target']
            labels = labels.type(torch.LongTensor)

            # forward + backward + optimize
            outputs = net(inputs.to(device))
            loss = criterion(outputs, torch.LongTensor(labels).to(device))

            pred = outputs.max(1, keepdim=True)[1].to('cpu')
            correct += pred.eq(labels.view_as(pred)).sum().item()
            running_loss += loss.item()
    print('[%d] test_acc: %.3f test_loss: %.3f' %(epoch + 1, correct/len(test_set), running_loss/len(test_set)))
    one_layer_gnet_acc_test.append(correct/len(test_set))
    one_layer_gnet_loss_test.append(running_loss/len(test_set))      

print('Finished Training')

np.save('AlexNet_acc_train.npy', one_layer_gnet_acc_train)
np.save('AlexNet_loss_train.npy', one_layer_gnet_loss_train)
np.save('AlexNet_acc_test.npy', one_layer_gnet_acc_test)
np.save('AlexNet_loss_test.npy', one_layer_gnet_loss_test)

In [ ]:
correct = 0
for i, data in enumerate(test):
    if i == 1:
        break
    # get the inputs
    inputs, labels = data['image'], data['target']
    labels = labels.type(torch.LongTensor)

    # forward + backward + optimize
    outputs = net(inputs.to(device))
    loss = criterion(outputs, labels.to(device))

    pred = outputs.max(1, keepdim=True)[1].to('cpu')
    correct += pred.eq(labels.view_as(pred)).sum().item()
print('labels: {}'.format(labels))
print('pred: {}'.format(pred.reshape(-1)))
print('correct: {}'.format(correct))

In [ ]:
for i in range(len(inputs)):
    if labels[i] == 7:
        plt.imshow(inputs[i][0].numpy())
        plt.show()

In [ ]:
print(inputs.mean())

In [ ]:
test_set.target.count(7)

In [ ]:
train_set.target.count(1)

In [ ]:
from skimage import io
image = io.imread('D:\\AffectivNet\\IMAGES\\1\\0da9ef7c8f852bac98b57d03926465c7312bf0d9accf1a4d56667112.jpg')